## Step 2: Data Analysis / KPI Calculations

Using the cleaned survey dataset, I calculated the key performance indicators (KPIs) required in the assessment.  
These KPIs summarize access to services, satisfaction levels, and demographic comparisons.

**KPIs computed:**
1. Average Q1 (“Access to Basic Health Services”) per Region  
2. Age Group with the lowest average Q1 score  
3. % of respondents who rated “5” for Q2 (Food Security)  
4. Survey Type with the highest overall satisfaction (average Q1–Q4)  
5. Comparison of Male vs Female average Q3 scores (Children attending school safely)  
6. Correlation between Q1 and Q2 (Health vs Food Security)


In [29]:

# STEP 2: DATA ANALYSIS / KPI CALCULATIONS


import pandas as pd
import numpy as np

# Load cleaned dataset

file_path = r"C:\Users\hp\Downloads\Cleaned_TL20251023Mock_Survey.xlsx"
df = pd.read_excel(file_path)

In [30]:
print("Dataset loaded successfully!")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

Dataset loaded successfully!
Rows: 123, Columns: 11


In [31]:
# Simplify and clean column names
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(":", "", regex=False)
)

In [34]:
# Automatically rename long Q columns to short labels

for col in df.columns.copy():
    if col.startswith("q1"):
        df.rename(columns={col: "q1"}, inplace=True)
    elif col.startswith("q2"):
        df.rename(columns={col: "q2"}, inplace=True)
    elif col.startswith("q3"):
        df.rename(columns={col: "q3"}, inplace=True)
    elif col.startswith("q4"):
        df.rename(columns={col: "q4"}, inplace=True)
    elif col.startswith("q5"):
        df.rename(columns={col: "q5"}, inplace=True)
        print("\n Final column names:")
print(df.columns.tolist())


 Final column names:
['submission_date', 'region', 'survey_type', 'respondent_id', 'gender', 'age_group', 'q1', 'q2', 'q3', 'q4', 'q5']


In [35]:
# Identify question columns automatically

q_cols = [c for c in df.columns if c.startswith("q")]
print("\nDetected question columns:", q_cols)



Detected question columns: ['q1', 'q2', 'q3', 'q4', 'q5']


In [48]:
#1: Calculate the average Q1 score (“Access to basic health services”) per Region.


# Average Q1 (“Access to Basic Health Services”) per Region

avg_q1_region = (
    df.groupby("region")["q1"]
    .mean()
    .reset_index()
    .sort_values("q1", ascending=False)
)
print("\n Average Q1 by Region:")
display(avg_q1_region)


 Average Q1 by Region:


,region,q1
3,South Asia,3.468750
4,West Africa,3.000000
2,Middle East,2.826087
0,East Africa,2.666667
1,Latin America,2.629630


In [49]:
# Find which Age Group reports the lowest access to health services.


# Age Group with lowest access to health services


avg_q1_age = df.groupby("age_group")["q1"].mean().reset_index().sort_values("q1")
lowest_age_group = avg_q1_age.iloc[0]
print(
    f"\n Age group with lowest access (Q1): "
    f"{lowest_age_group['age_group']} — Avg Q1 = {lowest_age_group['q1']:.2f}"
)
display(avg_q1_age)


 Age group with lowest access (Q1): 55+ — Avg Q1 = 2.56


,age_group,q1
4,55+,2.555556
2,35-44,2.620690
3,45-54,3.000000
0,18-24,3.111111
1,25-34,3.173913
5,Under 18,3.352941


In [50]:
# Determine the percentage of respondents who rated “5” for Q2 (Food security) across all regions.


# % respondents who rated “5” for Q2 (Food Security)

percent_q2_5 = (df["q2"] == 5).sum() / len(df) * 100
print(f"\n Percentage rating 5 for Q2 (Food Security): {percent_q2_5:.2f}%")


 Percentage rating 5 for Q2 (Food Security): 14.63%


In [51]:
# 4. Identify which Survey Type has the highest overall satisfaction (average across Q1–Q4).

# Survey Type with highest overall satisfaction (Avg Q1–Q4)

df["avg_q1_q4"] = df[["q1", "q2", "q3", "q4"]].mean(axis=1)
survey_avg = (
    df.groupby("survey_type")["avg_q1_q4"]
    .mean()
    .reset_index()
    .sort_values("avg_q1_q4", ascending=False)
)
top_survey = survey_avg.iloc[0]
print(
    f"\n Survey Type with highest satisfaction: "
    f"{top_survey['survey_type']} — Avg Q1–Q4 = {top_survey['avg_q1_q4']:.2f}"
)
display(survey_avg)


 Survey Type with highest satisfaction: Shelter — Avg Q1–Q4 = 3.13


,survey_type,avg_q1_q4
8,Shelter,3.131579
6,Health Access,3.102273
0,Ed Ucation,3.000000
5,He Alth Access,3.000000
9,Wa Ter & Sanitation,2.906250
3,Fo Od Security,2.875000
4,Food Security,2.809524
2,Education,2.808824
10,Water & Sanitation,2.708333
1,Ed Ucation,2.687500


In [52]:
# 5. Compare male vs. female respondents’ average scores in Q3 (Children attending school safely).

# Gender comparison for Q3 (Children attending school safely)

avg_q3_gender = df.groupby("gender")["q3"].mean().reset_index().sort_values("q3", ascending=False)
print("\n🎓 Average Q3 (Children attending school safely) by Gender:")
display(avg_q3_gender)



🎓 Average Q3 (Children attending school safely) by Gender:


,gender,q3
3,Prefer Not To Say,3.392857
1,Male,3.045455
0,Female,2.911765
2,Non-Binary,2.871795


In [53]:
# 6. Compute the correlation between access to health services (Q1) and food security (Q2).

# Correlation between Q1 and Q2 (Access to Health vs Food Security)

correlation_q1_q2 = df["q1"].corr(df["q2"])
print(f"\n Correlation between Q1 and Q2: {correlation_q1_q2:.4f}")


 Correlation between Q1 and Q2: 0.0493


In [55]:
# Save KPI 

output_path = r"C:\Users\hp\Downloads\KPI_Results_TL20251023Mock_Survey.xlsx"
with pd.ExcelWriter(output_path) as writer:
    avg_q1_region.to_excel(writer, sheet_name="Avg_Q1_by_Region", index=False)
    avg_q1_age.to_excel(writer, sheet_name="Avg_Q1_by_AgeGroup", index=False)
    survey_avg.to_excel(writer, sheet_name="Survey_Satisfaction", index=False)
    avg_q3_gender.to_excel(writer, sheet_name="Avg_Q3_by_Gender", index=False)

print(f"\n  KPI results saved to: {output_path}")


  KPI results saved to: C:\Users\hp\Downloads\KPI_Results_TL20251023Mock_Survey.xlsx
